In [1]:
# pre reqs 
"""
 zt = 2xt/vsound 
 ut = xt.. 
 xt = xt0 + ut(t)^2/2 
 ept = N(0,R)
 delt = N(0,Q) 
 X = [xt,xt.] 
 
 326 time stamps with 0.01 interval 
 xt(0)= 0 , var = 1e-4I 
 Q = 0.01 
 var(x) = 0.1^2 , var(x.) = 0.5^2 , R is diag 
 vsound = 3000 km/hr 
 u(t) = ... 
"""
import numpy as np 
import math 
tol = 1e-12

In [2]:
# True x,v 
def u(t):
    if t < 0.25:
        return 400 
    elif t > 3 and t < 3.25:
        return -400 
    else:
        return 0 

def v(t):
    if t < 0.25: 
        return u(t)*t 
    elif t > 3 and t < 3.25:
        return 100+u(t)*(t-3)
    else:
        return 0.25*u(0) 

def x(t):
    if t < 0.25:
        return (1/2)*u(t)*t**2 
    elif t > 3 and t <= 3.25:
        return (1/2)*u(0)*(0.25)**2 + (2.75)*v(0.25) + v(0.25)*(t-3) + (1/2)*u(t)*(t-3)**2
    else:
        return (1/2)*u(0)*(0.25)**2 + (t-0.25)*v(0.25)


In [3]:
# estimated x,v 
sig_x = 0.1 
sig_v = 0.5
sig_s = 0.01
time_step = 0.01
N = 326
v_sound = 3000
time = [(0.01*i-tol) for i in range(0,326)]

mu_0 = np.transpose(np.array([0,0]))
mu_0 = mu_0[:,np.newaxis]
sigma_0 = 1e-4*np.array([[1,0],[0,1]])
 
R = np.array([[sig_x**2,0],[0,sig_v**2]])
Q = np.array([[sig_s**2]])

A_t = np.array([[1,time_step],[0,1]]) 
B_t = np.transpose(np.array([0.5*time_step**2,time_step]))
B_t = B_t[:,np.newaxis]

C_t = np.array([[2/v_sound,0]]) 


def kalman_filter(mu_prev,sigma_prev,u_t,z_t):

    mu_t = A_t@mu_prev + B_t@u_t 
    sigma_t = A_t@sigma_prev + R 
    
    K_t = sigma_t@np.transpose(C_t)@np.linalg.inv((C_t@sigma_t@np.transpose(C_t)+Q))
    mu_t = mu_t + K_t@(z_t-C_t@mu_t)
    sigma_t = (np.array([[1,0],[0,1]])-K_t@C_t)@sigma_t
    return mu_t,sigma_t,K_t

from numpy.random import multivariate_normal 

x_true = [x(i) for i in time]
v_true = [v(i) for i in time]  

z_t = [(C_t@np.transpose(np.array([x_true[i],v_true[i]]))+multivariate_normal([0],Q,1)[0]) for i in range(len(x_true))]
K_t = []
mu_t = [mu_0]
sigma_t = [sigma_0]


for i,t in enumerate(time):
    mu,sigma,K = kalman_filter(mu_t[-1],sigma_t[-1],np.array([[u(t)]]),z_t[i]) 
    K_t.append(K)
    mu_t.append(mu)
    sigma_t.append(sigma) 



    
    
x_est = [mu[0][0] for mu in mu_t]
v_est = [mu[1][0] for mu in mu_t]   
x_est = x_est[:-1]
v_est= v_est[:-1] 
print(x_true)
print(x_est)
diff = [x_est[i]-x_true[i] for i in range(len(time))]
print(diff)
    
    
    


[1.9999999999999999e-22, 0.019999999996, 0.07999999999200001, 0.179999999988, 0.319999999984, 0.4999999999800001, 0.719999999976, 0.9799999999719999, 1.2799999999679998, 1.6199999999639996, 1.99999999996, 2.4199999999559996, 2.8799999999519996, 3.3799999999479997, 3.9199999999440003, 4.499999999939999, 5.119999999936, 5.779999999932, 6.479999999927999, 7.219999999924, 7.99999999992, 8.819999999916, 9.679999999911999, 10.579999999908, 11.519999999903998, 12.4999999999, 13.499999999900004, 14.499999999900004, 15.499999999900005, 16.499999999899998, 17.4999999999, 18.4999999999, 19.4999999999, 20.499999999900005, 21.499999999900005, 22.499999999900005, 23.4999999999, 24.4999999999, 25.4999999999, 26.499999999900005, 27.499999999900005, 28.499999999900005, 29.4999999999, 30.4999999999, 31.4999999999, 32.4999999999, 33.499999999900005, 34.499999999900005, 35.4999999999, 36.4999999999, 37.4999999999, 38.4999999999, 39.499999999900005, 40.499999999900005, 41.499999999900005, 42.49999999990000

In [4]:
import plotly.express as px
fig = px.scatter(x=time, y=x_true)
fig.add_scatter(x=time,y=v_true)
fig.add_scatter(x=time,y=x_est)
fig.add_scatter(x=time,y=v_est)
fig.show()
fig.write_html("r.html")

In [5]:
print(len(time))
print(len(x_est))
fig2 = px.scatter(x=time,y=x_est)
fig2.show()

326
326
